# Graph Classification

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader

import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd

import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max

from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from dataclasses import dataclass
from typing import Optional

import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor

from typing import List, Optional, Tuple, Union
import math
import torch
from torch import Tensor
from torch_geometric.nn.models.mlp import Linear
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import BatchNorm
import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_mincut_pool

/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

EPS = 1e-15


def dense_hoscpool(
    x,
    adj,
    s,
    mu=0.1,
    alpha=0.5,
    new_ortho=False,
    mask=None,
):
    r"""The highe-order pooling operator (HoscPool) from the paper
    `"Higher-order clustering and pooling for Graph Neural Networks"
    <http://arxiv.org/abs/2209.03473>`_. Based on motif spectral clustering,
    it captures and combines different levels of higher-order connectivity
    patterns when coarsening the graph.
    .. math::
        \mathbf{X}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{X}
        \mathbf{A}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{A} \cdot \mathrm{softmax}(\mathbf{S})
    based on the learned cluster assignment matrix :math:`\mathbf{S} \in \mathbb{R}^{B
    \times N \times K}`. This function returns the pooled feature matrix, the coarsened
    symmetrically normalised adjacency matrix, the motif spectral clustering loss :math:`\mathcal{L}_{mc}`
    and the orthogonality loss :math:`\mathcal{L}_{o}`.
    .. math::
        \mathcal{L}_{mc} &= - \frac{\alpha_1}{K} \cdot \text{Tr}\bigg(\frac{\mathbf{S}^\top \mathbf{A} \mathbf{S}}
            {\mathbf{S}^\top\mathbf{D}\mathbf{S}}\bigg) - \frac{\alpha_2}{K} \cdot \text{Tr}\bigg(
                \frac{\mathbf{S}^\top\mathbf{A}_{M}\mathbf{S}}{\mathbf{S}^\top\mathbf{D}_{M}\mathbf{S}}\bigg).
        \mathcal{L}_o &= \frac{1}{\sqrt{K}-1} \bigg( \sqrt{K} - \frac{1}{\sqrt{N}}\sum_{j=1}^K ||S_{*j}||_F\bigg)
    Args:
        x (Tensor): Node feature tensor :math:`\mathbf{X} \in \mathbb{R}^{B
            \times N \times F}` with batch-size :math:`B`, (maximum)
            number of nodes :math:`N` for each graph, and feature dimension
            :math:`F`.
        adj (Tensor): adjacency matrix :math:`\mathbf{A} \in \mathbb{R}^{B \times N \times N}`.
        s (Tensor): the learnable cluster assignment matrix :math:`\mathbf{S} \in \mathbb{R}^{B
            \times N \times K}` with number of clusters :math:`K`. The softmax
            does not have to be applied beforehand, since it is executed
            within this method.
        mu (Tensor, optional): scalar that controls the importance given to regularization loss
        alpha (Tensor, optional): scalar in [0,1] controlling the importance granted
            to higher-order information (in loss function).
        new_ortho (BoolTensor, optional): either to use new proposed loss or old one
        mask (BoolTensor, optional): Mask matrix
            :math:`\mathbf{M} \in {\{ 0, 1 \}}^{B \times N}` indicating
            the valid nodes for each graph. (default: :obj:`None`)
    :rtype: (:class:`Tensor`, :class:`Tensor`, :class:`Tensor`,
        :class:`Tensor`)
    """
    x = x.unsqueeze(0) if x.dim() == 2 else x
    adj = adj.unsqueeze(0) if adj.dim() == 2 else adj
    s = s.unsqueeze(0) if s.dim() == 2 else s

    (batch_size, num_nodes, _), k = x.size(), s.size(-1)

    s = torch.softmax(s, dim=-1)

    if mask is not None:
        mask = mask.view(batch_size, num_nodes, 1).to(x.dtype)
        x, s = x * mask, s * mask

    # Output adjacency and feature matrices
    out = torch.matmul(s.transpose(1, 2), x)
    out_adj = torch.matmul(torch.matmul(s.transpose(1, 2), adj), s)

    # Motif adj matrix - not sym. normalised
    motif_adj = torch.mul(torch.matmul(adj, adj), adj)
    motif_out_adj = torch.matmul(torch.matmul(s.transpose(1, 2), motif_adj), s)

    mincut_loss = ho_mincut_loss = 0
    # 1st order MinCUT loss
    if alpha < 1:
        diag_SAS = torch.einsum("ijj->ij", out_adj.clone())
        d_flat = torch.einsum("ijk->ij", adj.clone())
        d = _rank3_diag(d_flat)
        sds = torch.matmul(torch.matmul(s.transpose(1, 2), d), s)
        diag_SDS = torch.einsum("ijk->ij", sds) + EPS
        mincut_loss = -torch.sum(diag_SAS / diag_SDS, axis=1)
        mincut_loss = 1 / k * torch.mean(mincut_loss)

    # Higher order cut
    if alpha > 0:
        diag_SAS = torch.einsum("ijj->ij", motif_out_adj)
        d_flat = torch.einsum("ijk->ij", motif_adj)
        d = _rank3_diag(d_flat)
        diag_SDS = (torch.einsum(
            "ijk->ij", torch.matmul(torch.matmul(s.transpose(1, 2), d), s)) +
                    EPS)
        ho_mincut_loss = -torch.sum(diag_SAS / diag_SDS, axis=1)
        ho_mincut_loss = 1 / k * torch.mean(ho_mincut_loss)

    # Combine ho and fo mincut loss.
    # We do not learn these coefficients yet
    hosc_loss = (1 - alpha) * mincut_loss + alpha * ho_mincut_loss

    # Orthogonality loss
    if mu == 0:
        ortho_loss = torch.tensor(0)
    else:
        if new_ortho:
            if s.shape[0] == 1:
                ortho_loss = ((-torch.sum(torch.norm(s, p="fro", dim=-2)) /
                               (num_nodes**0.5)) + k**0.5) / (k**0.5 - 1)
            elif mask != None:
                ortho_loss = sum([((-torch.sum(
                    torch.norm(
                        s[i][:mask[i].nonzero().shape[0]],
                        p="fro",
                        dim=-2,
                    )) / (mask[i].nonzero().shape[0]**0.5) + k**0.5) /
                                   (k**0.5 - 1)) for i in range(batch_size)
                                  ]) / float(batch_size)
            else:
                ortho_loss = sum(
                    [((-torch.sum(torch.norm(s[i], p="fro", dim=-2)) /
                       (num_nodes**0.5) + k**0.5) / (k**0.5 - 1))
                     for i in range(batch_size)]) / float(batch_size)
        else:
            # Orthogonality regularization.
            ss = torch.matmul(s.transpose(1, 2), s)
            i_s = torch.eye(k).type_as(ss)
            ortho_loss = torch.norm(
                ss / torch.norm(ss, dim=(-1, -2), keepdim=True) -
                i_s / torch.norm(i_s),
                dim=(-1, -2),
            )
            ortho_loss = torch.mean(ortho_loss)

    # Fix and normalize coarsened adjacency matrix.
    ind = torch.arange(k, device=out_adj.device)
    out_adj[:, ind, ind] = 0
    d = torch.einsum("ijk->ij", out_adj)
    d = torch.sqrt(d + EPS)[:, None]
    out_adj = (out_adj / d) / d.transpose(1, 2)

    return out, out_adj, hosc_loss, mu * ortho_loss


def _rank3_diag(x):
    eye = torch.eye(x.size(1)).type_as(x)
    out = eye * x.unsqueeze(2).expand(*x.size(), x.size(1))
    return out

### PROTEINS

In [5]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 700
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="PROTEINS",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.5, alpha=0.5, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.5, alpha=0.5, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=128, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=128, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=128, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')


Seed: 42, Epoch: 001, Loss: 0.6825, Val Acc: 0.6988, Test Acc: 0.7440
Seed: 42, Epoch: 002, Loss: 0.6411, Val Acc: 0.6687, Test Acc: 0.7143
Seed: 42, Epoch: 003, Loss: 0.6301, Val Acc: 0.6325, Test Acc: 0.6905
Seed: 42, Epoch: 004, Loss: 0.6293, Val Acc: 0.6325, Test Acc: 0.6964
Seed: 42, Epoch: 005, Loss: 0.6218, Val Acc: 0.6687, Test Acc: 0.7143
Seed: 42, Epoch: 006, Loss: 0.6206, Val Acc: 0.6807, Test Acc: 0.7202
Seed: 42, Epoch: 007, Loss: 0.6173, Val Acc: 0.6747, Test Acc: 0.7202
Seed: 42, Epoch: 008, Loss: 0.6145, Val Acc: 0.6566, Test Acc: 0.7143
Seed: 42, Epoch: 009, Loss: 0.6099, Val Acc: 0.6506, Test Acc: 0.7024
Seed: 42, Epoch: 010, Loss: 0.6068, Val Acc: 0.6566, Test Acc: 0.7024
Seed: 42, Epoch: 011, Loss: 0.6039, Val Acc: 0.6627, Test Acc: 0.7143
Seed: 42, Epoch: 012, Loss: 0.6014, Val Acc: 0.6928, Test Acc: 0.7083
Seed: 42, Epoch: 013, Loss: 0.5967, Val Acc: 0.6627, Test Acc: 0.7083
Seed: 42, Epoch: 014, Loss: 0.5941, Val Acc: 0.6506, Test Acc: 0.7143
Seed: 42, Epoch: 015

### NCI1

In [6]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 150
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="NCI1",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.3, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.3, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=128, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=128, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=128, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6875, Val Acc: 0.4838, Test Acc: 0.4619
Seed: 42, Epoch: 002, Loss: 0.6634, Val Acc: 0.6266, Test Acc: 0.6224
Seed: 42, Epoch: 003, Loss: 0.6345, Val Acc: 0.6526, Test Acc: 0.6126
Seed: 42, Epoch: 004, Loss: 0.6105, Val Acc: 0.6331, Test Acc: 0.6321
Seed: 42, Epoch: 005, Loss: 0.6011, Val Acc: 0.6705, Test Acc: 0.6467
Seed: 42, Epoch: 006, Loss: 0.5928, Val Acc: 0.6169, Test Acc: 0.6434
Seed: 42, Epoch: 007, Loss: 0.5823, Val Acc: 0.6818, Test Acc: 0.6548
Seed: 42, Epoch: 008, Loss: 0.5669, Val Acc: 0.6851, Test Acc: 0.6840
Seed: 42, Epoch: 009, Loss: 0.5692, Val Acc: 0.7029, Test Acc: 0.6904
Seed: 42, Epoch: 010, Loss: 0.5673, Val Acc: 0.6705, Test Acc: 0.6791
Seed: 42, Epoch: 011, Loss: 0.5584, Val Acc: 0.6883, Test Acc: 0.6759
Seed: 42, Epoch: 012, Loss: 0.5503, Val Acc: 0.7208, Test Acc: 0.7245
Seed: 42, Epoch: 013, Loss: 0.5423, Val Acc: 0.6899, Test Acc: 0.7099
Seed: 42, Epoch: 014, Loss: 0.5395, Val Acc: 0.6786, Test Acc: 0.6807
Seed: 42, Epoch: 015

### NCI109

In [7]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 150
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="NCI109",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.3, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.3, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=128, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=128, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=128, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6878, Val Acc: 0.4879, Test Acc: 0.5081
Seed: 42, Epoch: 002, Loss: 0.6681, Val Acc: 0.6171, Test Acc: 0.5887
Seed: 42, Epoch: 003, Loss: 0.6427, Val Acc: 0.6414, Test Acc: 0.6113
Seed: 42, Epoch: 004, Loss: 0.6275, Val Acc: 0.6914, Test Acc: 0.6548
Seed: 42, Epoch: 005, Loss: 0.6128, Val Acc: 0.6979, Test Acc: 0.6403
Seed: 42, Epoch: 006, Loss: 0.5982, Val Acc: 0.7141, Test Acc: 0.6790
Seed: 42, Epoch: 007, Loss: 0.5881, Val Acc: 0.6898, Test Acc: 0.6710
Seed: 42, Epoch: 008, Loss: 0.5887, Val Acc: 0.6801, Test Acc: 0.6468
Seed: 42, Epoch: 009, Loss: 0.5875, Val Acc: 0.6834, Test Acc: 0.6274
Seed: 42, Epoch: 010, Loss: 0.5803, Val Acc: 0.6866, Test Acc: 0.6726
Seed: 42, Epoch: 011, Loss: 0.5765, Val Acc: 0.7286, Test Acc: 0.6677
Seed: 42, Epoch: 012, Loss: 0.5673, Val Acc: 0.7141, Test Acc: 0.6742
Seed: 42, Epoch: 013, Loss: 0.5635, Val Acc: 0.7011, Test Acc: 0.6823
Seed: 42, Epoch: 014, Loss: 0.5655, Val Acc: 0.7173, Test Acc: 0.6839
Seed: 42, Epoch: 015

### MUTAG

In [8]:
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import BaseTransform

from torch_geometric.utils import to_dense_adj
data_path = "/data/XXX/Pooling"
max_nodes = 150
class ConvertToDenseAdj(BaseTransform):
    def __call__(self, data):
        # 确保 data.adj 存在且为三维
        if hasattr(data, 'adj') and data.adj.dim() == 3:
            # 对第三维进行合并操作，这里以求和为例
            data.adj = data.adj.sum(dim=-1)
            # 你可以选择其他方式，如取最大值：
            # data.adj = data.adj.max(dim=-1)[0]

        return data

# 在加载数据时应用这个变换
dataset_dense = TUDataset(
    data_path,
    name="MUTAG",
    transform=T.Compose([T.ToDense(max_nodes), ConvertToDenseAdj()]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
dataset_dense[3]

Data(x=[150, 7], y=[1], adj=[150, 150], mask=[150])

In [9]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool


class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.5, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.5, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 34.75 seconds
Var Time: 73.19 seconds
Average Memory: 428.67 MB
Average Best Val Acc: 0.9048
Std Best Test Acc: 0.0163
Average Test Acc: 0.8506


### DD

In [10]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 500
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="DD",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.1, alpha=0.5, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.5, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6906, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 002, Loss: 0.6889, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 003, Loss: 0.6877, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 004, Loss: 0.6868, Val Acc: 0.5495, Test Acc: 0.5676
Seed: 42, Epoch: 005, Loss: 0.6861, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 006, Loss: 0.6854, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 007, Loss: 0.6847, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 008, Loss: 0.6840, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 009, Loss: 0.6831, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 010, Loss: 0.6820, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 011, Loss: 0.6808, Val Acc: 0.5766, Test Acc: 0.5766
Seed: 42, Epoch: 012, Loss: 0.6793, Val Acc: 0.6937, Test Acc: 0.6667
Seed: 42, Epoch: 013, Loss: 0.6775, Val Acc: 0.7297, Test Acc: 0.6937
Seed: 42, Epoch: 014, Loss: 0.6752, Val Acc: 0.7748, Test Acc: 0.7387
Seed: 42, Epoch: 015

### IMDB-BINARY

In [8]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="IMDB-BINARY",
    transform=T.Compose([T.OneHotDegree(136), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.1, alpha=0.7, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.1, alpha=0.7, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6961, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 002, Loss: 0.6954, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 003, Loss: 0.6949, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 004, Loss: 0.6943, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 005, Loss: 0.6937, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 006, Loss: 0.6930, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 007, Loss: 0.6921, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 008, Loss: 0.6909, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 009, Loss: 0.6895, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 010, Loss: 0.6874, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 011, Loss: 0.6845, Val Acc: 0.5267, Test Acc: 0.5533
Seed: 42, Epoch: 012, Loss: 0.6815, Val Acc: 0.5267, Test Acc: 0.5667
Seed: 42, Epoch: 013, Loss: 0.6778, Val Acc: 0.5333, Test Acc: 0.6333
Seed: 42, Epoch: 014, Loss: 0.6730, Val Acc: 0.5933, Test Acc: 0.6867
Seed: 42, Epoch: 015

### IMDB-MULTI

In [11]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="IMDB-MULTI",
    transform=T.Compose([T.OneHotDegree(88), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.1, alpha=0.7, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.1, alpha=0.7, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.0996, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 002, Loss: 1.0986, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 003, Loss: 1.0980, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 004, Loss: 1.0974, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 005, Loss: 1.0967, Val Acc: 0.2711, Test Acc: 0.3511
Seed: 42, Epoch: 006, Loss: 1.0960, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 007, Loss: 1.0950, Val Acc: 0.2711, Test Acc: 0.3600
Seed: 42, Epoch: 008, Loss: 1.0940, Val Acc: 0.2711, Test Acc: 0.3644
Seed: 42, Epoch: 009, Loss: 1.0929, Val Acc: 0.2844, Test Acc: 0.3733
Seed: 42, Epoch: 010, Loss: 1.0914, Val Acc: 0.2889, Test Acc: 0.3778
Seed: 42, Epoch: 011, Loss: 1.0894, Val Acc: 0.2844, Test Acc: 0.3778
Seed: 42, Epoch: 012, Loss: 1.0870, Val Acc: 0.3022, Test Acc: 0.3778
Seed: 42, Epoch: 013, Loss: 1.0840, Val Acc: 0.3244, Test Acc: 0.4000
Seed: 42, Epoch: 014, Loss: 1.0802, Val Acc: 0.3556, Test Acc: 0.4400
Seed: 42, Epoch: 015

### COLLAB

In [12]:

max_nodes = 500
data_path = "/data/XXX/Pooling"

dataset_dense = TUDataset(
    data_path,
    name="COLLAB",
    transform=T.Compose([T.OneHotDegree(491), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())

        return x


class Net_hosc(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)

        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)

        self.gnn3_embed = DenseGCNConv(64, 64)

        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)


    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)

        x, adj, mc, o = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.5, new_ortho=False, mask=mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)

        x, adj, mc_aux, o_aux = dense_hoscpool(x, adj, s, mu=0.3, alpha=0.5, new_ortho=False)

        x = self.gnn3_embed(x, adj)
        x = F.relu(x)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Net_hosc().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)

    model = Net_hosc().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.1240, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 002, Loss: 1.1014, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 003, Loss: 1.0578, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 004, Loss: 0.9834, Val Acc: 0.4293, Test Acc: 0.4653
Seed: 42, Epoch: 005, Loss: 0.8906, Val Acc: 0.6400, Test Acc: 0.6387
Seed: 42, Epoch: 006, Loss: 0.7986, Val Acc: 0.6053, Test Acc: 0.6227
Seed: 42, Epoch: 007, Loss: 0.7194, Val Acc: 0.6240, Test Acc: 0.6373
Seed: 42, Epoch: 008, Loss: 0.6428, Val Acc: 0.6440, Test Acc: 0.6520
Seed: 42, Epoch: 009, Loss: 0.5863, Val Acc: 0.6587, Test Acc: 0.6707
Seed: 42, Epoch: 010, Loss: 0.5544, Val Acc: 0.6813, Test Acc: 0.6827
Seed: 42, Epoch: 011, Loss: 0.5312, Val Acc: 0.6800, Test Acc: 0.6880
Seed: 42, Epoch: 012, Loss: 0.5124, Val Acc: 0.7107, Test Acc: 0.6987
Seed: 42, Epoch: 013, Loss: 0.4944, Val Acc: 0.7160, Test Acc: 0.7013
Seed: 42, Epoch: 014, Loss: 0.4817, Val Acc: 0.7213, Test Acc: 0.7053
Seed: 42, Epoch: 015

# Graph Regression

### QM7

In [3]:
!python /data/XXX/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --pooling='Hosc'

Existed dataset loaded: datasets/processed/qm7.pt

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500MAE=1545.3601 MAE=1462.0270 MAE=1374.7219 MAE=1404.1001 MAE=1220.8877 MAE=1352.8748 MAE=1275.0078 MAE=1155.1167 MAE=1121.7788 Epoch: 10/500MAE=996.9196 MAE=876.5198 MAE=591.0928 MAE=533.2172 MAE=701.3717 MAE=379.4106 MAE=644.5787 MAE=554.1118 MAE=275.7677 MAE=210.0068 Epoch: 20/500MAE=272.9962 MAE=541.7998 MAE=731.6129 MAE=141.0365 MAE=112.0555 MAE=127.7508 MAE=78.6844 MAE=71.7933 MAE=76.6619 MAE=72.2410 Epoch: 30/500MAE=382.3224 MAE=190.6062 MAE=100.2302 MAE=90.9659 MAE=103.1871 MAE=61.7845 MAE=54.1816 MAE=124.0387 MAE=59.9093 MAE=72.5676 Epoch: 40/500MAE=84.4918 MAE=56.4927 MAE=51.8493 MAE=72.5265 MAE=52.0390 MAE=51.7178 MAE=75.9519 MAE=53.4465 MAE=55.1383 MAE=68.4591 Epoch: 50/500MAE=53.7295 MAE=54.7499 MAE=55.5125 MAE=51.8730 MAE=51.7146 MAE=51.1614 MAE=54.3166 MAE=51.2502 MAE=52.3182 MAE=50.9919 Epoch: 60/500MAE=49.7038 MAE=54.